# INSTALLS & IMPORTS

In [ ]:
#INSTALACION PAQUETE DEEPFOREST
!pip install DeepForest==0.3.8
!pip install tensorflow-gpu==1.14.0 # Para usar GPU
#es necesario desinstalar y reinstalar numpy por un problema de compatibilidad de versiones.
!pip uninstall numpy # PIDE METER UN YES OJO
!pip install numpy
# INSTALACIÓN PAQUETES TRATAR DATOS
!apt install gdal-bin python3-gdal
!apt-get install python-numpy python-scipy -y # Install numpy and scipy
!pip install pyshp # Shapefile
!pip install fiona==1.7
!pip install rasterio
!pip install pyproj
# INSTALACIÓN PAQUETES PARA ANALISIS RESULTADOS
!pip install plotly==4.14.3 # Instala en la máquina virtual Plot.ly
!pip install -U kaleido     # Instala la librería que exporta las figuras a imágenes
!pip install pyyaml==5.4.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.3 MB 5.9 MB/s 
     |████████████████████████████████| 2.9 MB 36.6 MB/s 
     |████████████████████████████████| 109.3 MB 44 kB/s 
     |████████████████████████████████| 596 kB 47.2 MB/s 
     |████████████████████████████████| 377 kB 47.2 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
     |████████████████████████████████| 488 kB 47.4 MB/s 
     |████████████████████████████████| 3.1 MB 11.7 MB/s 
  Created wheel for keras-resnet: filename=keras_resnet-0.1.0-py2.py3-none-any.whl size=13343 sha256=14e7e529b519de7667ca8f54da3562cb314e6c79bf571656837892bc6eb7000a
  Stored in directory: /root/.cache/pip/wheels/a6/20/57/d7b7be8556e2ebf36345b3df4068a9a13bb90e4a2cc85a6994
Successfully built keras-resnet
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5

In [ ]:
# IMPORTS PRCESO DEEPFOREST
# ANTES HAY QUE DARLE A RESTART RUNTIME
import numpy as np
import os
from PIL import Image
from matplotlib import pyplot as plt
from deepforest import deepforest
from deepforest import get_data
from deepforest import utilities
from deepforest import preprocess
import pandas as pd

# IMPORTS TRATAR CON DATOS
import gdal
import os
from osgeo import osr, ogr, gdal
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from skimage import io
from PIL import Image 
import matplotlib.pylab as plt
from osgeo import ogr
from shapely.geometry import Polygon
import math
!pip install fiona
import fiona
from fiona.crs import from_epsg
from shapely.geometry import MultiPolygon, Polygon, mapping
import matplotlib.pyplot as plt
from descartes import PolygonPatch
import os
import rasterio
import rasterio.mask
import csv
#IMPORTS ANALISIS DE RESULTADOS
# Plotly
#from IPython.display import Image # to display images
import plotly       # Check that Plot.ly version is correct (prev versions than 4.14.3 do not use Kaleido)
plotly.__version__
import plotly.graph_objects as go # Se importa la librería
import plotly.express as px                         # Cosas de plotly generales. Como los colores
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
import plotly.graph_objs as go              # Para generar las figuras y los charts
import plotly.io as pio                     # Interfaz para guardar en SVG
import math
from pyproj import Geod
from shapely import wkt
# Prepara los símbolos y colores
from plotly.validators.scatter.marker import SymbolValidator

#IMPORTS ACCESO A COLAB
from google.colab import output # Permite limpiar el log de Colab cada X tiempo
from google.colab import drive # acceso a ficheros de Drive
from google.colab import files # descargar ficheros generados
import glob     # to access Google Drive files
import os       # tratar nombres de ficheros y carpetas
drive.mount('/content/drive')
# Definir rutas
path      = "/content/drive/MyDrive/tree_detection/Net_Trained_With_LiDAR_Detections"   # Base. Según la cuenta será una u otra.  
# Cambia a la carpeta
%cd {path}
#!ls                # Muestra los ficheros para comprobar que la carpeta es correcta

tif_image      = 'Orthophoto_Colmenarejo_Train'               # Fichero original
fileName=tif_image
fileExtension  = '.tif'                 # Extensión del fichero

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/tree_detection/Net_Trained_With_LiDAR_Detections'
/content


# Read datasets SHP LiDAR

In [ ]:
#################
def world_to_pixel(geo_matrix, x, y):
    """
    Uses a gdal geomatrix (gdal.GetGeoTransform()) to calculate
    the pixel location of a geospatial coordinate
    """
    ul_x   = geo_matrix[0]
    ul_y   = geo_matrix[3]
    x_dist = geo_matrix[1]
    y_dist = geo_matrix[5]
    pixel  =  int((x - ul_x) / x_dist)
    line   = -int((ul_y - y) / y_dist)
    return pixel, line

#################
def pixel_to_world(geo_matrix, x, y):
    ul_x = geo_matrix[0]
    ul_y = geo_matrix[3]
    x_dist = geo_matrix[1]
    y_dist = geo_matrix[5]
    _x = x * x_dist + ul_x
    _y = y * y_dist + ul_y
    return _x, _y

#################
def build_transform_inverse(dataset, EPSG):
    source = osr.SpatialReference(wkt=dataset.GetProjection())
    target = osr.SpatialReference()
    target.ImportFromEPSG(EPSG)
    return osr.CoordinateTransformation(source, target)

#################
def find_spatial_coordinate_from_pixel(dataset, transform, x, y):
    world_x, world_y = pixel_to_world(dataset.GetGeoTransform(), x, y)
    point = ogr.Geometry(ogr.wkbPoint)
    point.AddPoint(world_x, world_y)
    point.Transform(transform)
    return point.GetX(), point.GetY()

#################
# La ortofoto tiene su lat-lon predefinido. 
# Esta función recibe una Lat Lon esquina arriba izq y abajo der y te devuelve qué píxeles correponden a dichos lugares
def getTIFcornerPoints(fileName, LatAI, LonAI, LatAD, LonAD):
    fileExtension = ".tif"
    dem = gdal.Open(fileName+fileExtension)
    xmin, res, rot1, ymax, px_h, rot2 = dem.GetGeoTransform() # get coordinates of upper left corner

    # Extraer el EPSG que modela las coordenadas
    EPSG   = osr.SpatialReference(wkt=dem.GetProjection()).GetAttrValue('AUTHORITY',1)
    source = osr.SpatialReference()
    source.ImportFromEPSG(int(EPSG))
    # Transformación a WGS84
    wgs84 = osr.SpatialReference()
    wgs84.ImportFromEPSG(4326)
    transSourWGS = osr.CoordinateTransformation(source, wgs84)
    transWGSSour = osr.CoordinateTransformation(wgs84, source)
    # transformando los puntos de los recortes a hacer
    xAI, yAI, _ = transWGSSour.TransformPoint(LonAI, LatAI) # arriba izq v2
    xAD, yAD, _ = transWGSSour.TransformPoint(LonAD, LatAD)  # abajo der v2

    #print(xAI, yAI, xAD, yAD)
    return xAI, yAI, xAD, yAD

#################
def cornerPointsToSHP(outputName, xAI, yAI, xAD, yAD, EPSG=4326):
    finalPolygonList = []
    polygon = Polygon([(xAI, yAI),
                      (xAI, yAD),
                      (xAD, yAD),
                      (xAD, yAI),
                      (xAI, yAI)])
    finalPolygonList.append(polygon)

    # Define a polygon feature geometry with one attribute
    polygonsToSHP(outputName, '.shp', finalPolygonList, EPSG=EPSG)

def polygonsToSHP(nameOutputSHP, SHPExtension, finalPolygonList, EPSG=4326):
    schema = {
        'geometry': 'Polygon',
        'properties': {'id': 'int'},
    }
    # Se crea un fichero SHP en el que se almacenan los polygon
    outFile = fiona.open(nameOutputSHP+SHPExtension, mode = 'w', crs=from_epsg(EPSG), driver='ESRI Shapefile', schema=schema)
    for index, poly in enumerate(finalPolygonList):
        outFile.write({
            'geometry':mapping(poly),
            'properties': {
                'id': index
            },
        })
    outFile.close()

####################
# Unir Shapefiles de detecciones LiDAR en uno solo
def mergeSHPs(nameOutputSHP, folderOrigSHP, SHPExtension=".shp"):
  # Open polygon layer and create list of with all the Polygons
  polyList = []
  polyProperties = []

  for filename in os.listdir(folderOrigSHP):
    if filename.endswith(SHPExtension):
      #print(filename)
      polyShp = fiona.open(folderOrigSHP+"/"+filename)
      for poly in polyShp:
        polyGeom = poly['geometry']['coordinates'][0]
        polyList.append(polyGeom)

  # Se crea la transformación a WGS84
  epsgSHP = polyShp.meta['crs']['init'] # Extraer el EPSG con el ultimo SHP (deberían ser todos iguales)
  epsgSHP = epsgSHP.replace("epsg:", "")
  source = osr.SpatialReference()
  source.ImportFromEPSG(int(epsgSHP))
  EPSG_wgs84 = 4326
  wgs84 = osr.SpatialReference()
  wgs84.ImportFromEPSG(EPSG_wgs84)
  transSourWGS = osr.CoordinateTransformation(source, wgs84)

  # Se cogen todos los polígonos de cada fichero
  finalPolygonList = []
  for poly in polyList:
    auxPoly = []
    for point in poly:
      Lon, Lat, _ = transSourWGS.TransformPoint(point[0],point[1])
      a = (Lon, Lat)
      auxPoly.append(a)
    # Se genera un polígono (que cierra) del árbol detectado en WGS84
    polygon = Polygon(auxPoly)
    finalPolygonList.append(polygon)

  # Export clipped polygons as shapefile
  polygonsToSHP(nameOutputSHP, SHPExtension, finalPolygonList, EPSG=EPSG_wgs84)

#################
# 
def TIFgetESPG(fileName):
  # Abro la imagen y extraigo su EPSG
  fileExtension = ".tif"
  dem = gdal.Open(fileName+fileExtension)
  # Extraer el EPSG que modela las coordenadas
  EPSG   = osr.SpatialReference(wkt=dem.GetProjection()).GetAttrValue('AUTHORITY',1)
  dem   = None
  return int(EPSG)

#################
def SHPgetESPG(SHPName, SHPExtension=".shp"):
  polyShp = fiona.open(SHPName+SHPExtension)
  epsgSHP = polyShp.meta['crs']['init']
  epsgSHP = epsgSHP.replace("epsg:", "")
  return int(epsgSHP)

#################
def cropTIFbySHP(TIFOriginalName, TIFOutputName, SHPName):
  EPSG = TIFgetESPG(TIFOriginalName)
  TIFExtension = ".tif"
  SHPExtension = ".shp"

  # Cargo el SHP que define la zona a cortar
  with fiona.open(SHPName+SHPExtension, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

  # Abro la imagen y la corto con lo definido
  with rasterio.open(TIFOriginalName+TIFExtension) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    a = 'EPSG:'+str(EPSG)
    out_meta.update({
      "driver": "GTiff",
      "height": out_image.shape[1],
      "width": out_image.shape[2],
      "transform": out_transform,
      "crs": a
    })

  # Guardo el corte
  with rasterio.open(TIFOutputName+TIFExtension, "w", **out_meta) as dest:
    dest.write(out_image)

#################
def findMinMaxLatLonSHP(SHPName, SHPExtension=".shp"):
  # Identificar LatLon esquinas de todos los Shapefile, para definir el area del recorte a realizar
  minLat = 999
  minLon = 999
  maxLat =-999
  maxLon =-999

  polyShp = fiona.open(SHPName)
  for poly in polyShp:
    polyGeom = poly['geometry']['coordinates'][0]
    for point in polyGeom:
      if minLat > point[1]:
        minLat = point[1]
      if point[1] > maxLat:
        maxLat = point[1]
      if minLon > point[0]:
        minLon = point[0]
      if point[0] > maxLon:
        maxLon = point[0]

  return minLat, minLon, maxLat, maxLon

#################
def TIFToPNG(filename):
  TIFExtension = ".tif"
  PNGExtension = ".png"
  img = cv2.imread(filename+TIFExtension)
  cv2.imwrite(filename+PNGExtension, img)

#################
def XYPolygonToCSV(CSVName, TIFName, polyList, TIFExtension=".tif", CSVExtension=".csv"):
  with open(CSVName+CSVExtension, mode='w') as csv_write:
    csv_write = csv.writer(csv_write, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_write.writerow(['image_path','xmin','ymin','xmax','ymax','label']) # Header
    for index, poly in enumerate(polyList):
      xAI = poly[0]
      yAI = poly[1]
      xAD = poly[2]
      yAD = poly[3]

      # Los invierte para que esten bien colocadas las BB
      if xAI > xAD:
        a = xAD
        b = xAI
        xAD = b
        xAI = a
      if yAI > yAD:
        a = yAD
        b = yAI
        yAD = b
        yAI = a
      csv_write.writerow([TIFName+TIFExtension, xAI, yAI, xAD, yAD, 'Tree'])

#################
def BBtoTIFpixel(polyList, SHPEPSG, TIFName, CSVName, TIFExtension=".tif", CSVExtension=".csv"):
  # Abro la foto en la que quiero extraer las posiciones
  dem = gdal.Open(TIFName+TIFExtension)
  EPSG = TIFgetESPG(TIFName)
  print("Imagen EPSG: ", str(EPSG))

  source = osr.SpatialReference()
  source.ImportFromEPSG(int(EPSG))
  EPSG_wgs84 = SHPEPSG
  wgs84 = osr.SpatialReference()
  wgs84.ImportFromEPSG(EPSG_wgs84)
  transSourWGS = osr.CoordinateTransformation(source, wgs84)
  transWGSSour = osr.CoordinateTransformation(wgs84, source)

  # Cada detección a pixel
  newPolyList = []
  for index, poly in enumerate(polyList): # Cada poligono
    #print(poly)
    newPolyX = []
    newPolyY = []
    for polyVertex in poly:             # cada vértice del polígono
      #print(polyVertex)
      # Lo transformo a las EPSG de la TIF
      xTIF, yTIF, _ = transWGSSour.TransformPoint(polyVertex[0], polyVertex[1])
      #print(xTIF, yTIF)
      # Hace lo mismo que lo de arriba
      #with rasterio.open(TIFName+TIFExtension) as src:
      #    rows, cols = rasterio.transform.rowcol(src.transform, x, y)
      # Y finalmente a Lat, Lon
      x, y = world_to_pixel(dem.GetGeoTransform(), xTIF, yTIF)
      #print(x, y)
      newPolyX.append(x)
      newPolyY.append(y)

    newPolyX = list(set(newPolyX)) # Pilla los unique
    newPolyY = list(set(newPolyY))

    if len(newPolyX) == 1: # Si solo hay 1, es que esta mal la BB
      continue
    if len(newPolyY) == 1:
      continue

    # Los coloca correctamente para una BB
    if newPolyX[0] > newPolyX[1]:
      xAI = newPolyX[1]
      xAD = newPolyX[0]
    else:
      xAI = newPolyX[0]
      xAD = newPolyX[1]
    if newPolyY[0] > newPolyY[1]:
      yAI = newPolyY[1]
      yAD = newPolyY[0]
    else:
      yAI = newPolyY[0]
      yAD = newPolyY[1]

    newPolyList.append([xAI, yAI, xAD, yAD])

  return newPolyList

#################
def SHPtoCSV(SHPName, TIFName, CSVName, SHPExtension=".shp", TIFExtension=".tif", CSVExtension=".csv"):
  print("SHPtoCSV")
  #open polygon layer and create list of Polygons
  SHP_ESPG = SHPgetESPG(SHPName, SHPExtension)
  polyList = SHPgetPolygons(SHPName, SHPExtension)
  print(len(polyList))
  print("SHP EPSG: ", str(SHP_ESPG))
  # Pasa los poligonos a XY de la imagen dada
  newPolyList = BBtoTIFpixel(polyList, SHP_ESPG, TIFName, CSVName, TIFExtension, CSVExtension)
  # Escritura a CSV¿?
  print(len(newPolyList))
  XYPolygonToCSV(CSVName, TIFName, newPolyList, TIFExtension, CSVExtension)

###################
def deepForestToSHP(trained_model_boxes, TIFName, SHPOutputName, SHPExtension=".shp", TIFExtension=".tif"):
  # Se transforman a WGS84
  EPSG_wgs84 = 4326
  ds = gdal.Open(TIFName+TIFExtension)
  _t = build_transform_inverse(ds, EPSG_wgs84) # WGS84

  # Generacion de los poligonos 
  finalPolygonList = []
  for index, BB in trained_model_boxes.iterrows():
    auxPoly = []
    xAI = BB["xmin"]
    xAD = BB["xmax"]
    yAI = BB["ymin"]
    yAD = BB["ymax"]

    coordinates = find_spatial_coordinate_from_pixel(ds, _t, xAI,yAI)
    auxPoly.append( coordinates )
    coordinates = find_spatial_coordinate_from_pixel(ds, _t, xAI,yAD)
    auxPoly.append( coordinates )
    coordinates = find_spatial_coordinate_from_pixel(ds, _t, xAD,yAD)
    auxPoly.append( coordinates )
    coordinates = find_spatial_coordinate_from_pixel(ds, _t, xAD,yAI)
    auxPoly.append( coordinates )
    coordinates = find_spatial_coordinate_from_pixel(ds, _t, xAI,yAI)
    auxPoly.append( coordinates )
    # Se genera un polígono (que cierra) del árbol detectado en WGS84
    polygon = Polygon(auxPoly)
    finalPolygonList.append(polygon)

  # Export clipped polygons as shapefile
  polygonsToSHP(SHPOutputName, SHPExtension, finalPolygonList, EPSG=EPSG_wgs84)

# Merge SHP LiDAR

In [ ]:
# JUNTA TODOS LOS SHP LIDAR (LOS ARBOLES DETECTADOS DE R) EN UN UNICO SHP PARA TENERLOS TODOS JUNTOS
# USO COLMENOUC3M PORQUE EN ALGUNA CARPETA ESTAN TODOS LOS SHP MENOS EL DEL CAMPUS

# Unión SHP SIN la zona de la UC3M
mergeSHPs("Train/Train", "LiDAR_Detections/LiDAR_Detections_for_Training/")

# Unión SHP CON la zona de la UC3M
mergeSHPs("Example/Example", "LiDAR_Detections/LiDAR_Detections_for_Example/")

# Clean LiDAR detections with OSM

In [ ]:
###########
# Limpieza de SHP si coinciden un porcentaje con otros SHP (edificios, carreteras...)
# Apunta los que eliminaría en un CSV, porque tarda mucho y lo mismo peta
###########
def cleanSHPOverlappingRoadCSV(cleanSHP, cleanPath, roadSHP, roadPath, SHPExtension=".shp", threshold=25):
  # Abre ambos SHP
  polygonsToClean = SHPgetPolygons(cleanPath+cleanSHP, SHPExtension)
  polygonsRoads   = SHPgetPolygons(roadPath+roadSHP, SHPExtension)
  polygonsRoads = polygonsRoads[0]
  cleanEPSG = SHPgetESPG(cleanPath+cleanSHP)
  roadEPSG  = SHPgetESPG(roadPath+roadSHP)

  # Crea un fichero donde va a apuntando por orden cuales elimina
  f = open(cleanPath+"DeletedRoadsDetections.txt", "a")
  for idx, polyClean in enumerate(polygonsToClean): # Por cada poligono a limpiar
    if idx <= 0:     # Para continuar desde donde terminó. COLOCARLO A MANO
      continue
    if idx%500 == 0:  # Para limpiar un poco el output del Colab y que no vaya como la mierda
      output.clear()
      f.close()       # Cierra para que se guarde en el CSV cada cierto tiempo
      f = open(cleanPath+"DeletedRoadsDetections.txt", "a")
    if idx%10 == 0:   # Para imprimir menos
      print(idx, "de", len(polygonsToClean))

    checkAllOSM = True
    p1 = polygonToWGS84(polyClean, cleanEPSG) # Se transforma de array y como estuviese, a WGS84 y a Polygon
    for polyRoads in polygonsRoads:           # Por cada carretera [0]
      p2 = polygonToWGS84(polyRoads[0], roadEPSG)
      overlapDetected   = getOverlapPercentage(p1, p2) # Interseca con la carretera
      if overlapDetected > threshold: # Si supera un threshold, este polígono al carrer
        intersectsRoad = True
        for idx2 in range(1,len(polyRoads)): # Se mira el resto de huecos del poligono de la carretera
          # se hace la intersección de la detección a este hueco
          p3 = polygonToWGS84(polyRoads[idx2], roadEPSG)
          overlapDetected2 = getOverlapPercentage(p1, p3) # Interseca con la carretera
          if overlapDetected2 > threshold: # Si supera un threshold, está más dentro del hueco que de la carretera, luego no se elimina
            intersectsRoad = False
            break
        # Al mirar todos los huecos de esta carretera, sigue dentro, se elimina
        if intersectsRoad == True:
          print("hecho a: ", idx)
          f.write(str(idx)+"\n")

###########
# Limpieza de SHP si coinciden un porcentaje con otros SHP (edificios, carreteras...)
# Escribe los polígonos MENOS LOS DEL CSV en un nuevo SHP
###########
def cleanSHPCSVtoSHP(cleanSHP, cleanPath, inputCSV, outputName, SHPExtension=".shp"):
  # Abre ambos SHP
  polygonsToClean = SHPgetPolygons(cleanPath+cleanSHP, SHPExtension)
  cleanEPSG       = SHPgetESPG(cleanPath+cleanSHP)
  # Lectura del txt
  array = []
  with open(cleanPath+inputCSV) as f:
    for line in f:
      #print(line)
      array.append(int(line))
  # Pilla todos los polygons
  newPolyList = []
  for idx, polyClean in enumerate(polygonsToClean): # Por cada poligono a limpiar
    if idx%5000 == 0:  # Para limpiar un poco el output del Colab y que no vaya como la mierda
      output.clear()
    if idx%500 == 0:   # Para imprimir menos
      print(idx, "de", len(polygonsToClean))
    p1 = polygonToWGS84(polyClean, cleanEPSG) # Se transforma de array y como estuviese, a WGS84 y a Polygon
    if not idx in array: # Si no ha entrado petado con ninguno, se escribe, no overlapea
      newPolyList.append(p1)
  # Escribe los polígonos almacenados en un SHP
  polygonsToSHP(cleanPath+outputName, SHPExtension, newPolyList, EPSG=4326)

###########
# Limpieza de SHP si coinciden un porcentaje con otros SHP (edificios, carreteras...)
# Apunta los que eliminaría en un CSV, porque tarda mucho y lo mismo peta
###########
def cleanSHPOverlappingBuildingCSV(cleanSHP, cleanPath, buildingsSHP, buildingsPath, SHPExtension=".shp", threshold=25):
  # Abre ambos SHP
  polygonsToClean   = SHPgetPolygons(cleanPath+cleanSHP, SHPExtension)
  polygonsBuildings = SHPgetPolygons(buildingsPath+buildingsSHP, SHPExtension)
  cleanEPSG     = SHPgetESPG(cleanPath+cleanSHP)
  buildingsEPSG = SHPgetESPG(buildingsPath+buildingsSHP)

  # Crea un fichero donde va a apuntando por orden cuales elimina
  f = open(cleanPath+"DeletedBuildingsDetections.txt", "a")
  for idx, polyClean in enumerate(polygonsToClean): # Por cada poligono a limpiar
    if idx <= 0:     # Para continuar desde donde terminó. COLOCARLO A MANO
      continue
    if idx%500 == 0:  # Para limpiar un poco el output del Colab y que no vaya como la mierda
      output.clear()
      f.close()       # Cierra para que se guarde en el CSV cada cierto tiempo
      f = open(cleanPath+"DeletedBuildingsDetections.txt", "a")
    if idx%10 == 0:   # Para imprimir menos
      print(idx, "de", len(polygonsToClean))

    p1 = polygonToWGS84(polyClean, cleanEPSG) # Se transforma de array y como estuviese, a WGS84 y a Polygon
    for polyBuildings in polygonsBuildings:         # Por cada edificio
      p2 = polygonToWGS84(polyBuildings, buildingsEPSG)
      overlapDetected   = getOverlapPercentage(p1, p2) # Se calcula el porcentaje de overlap
      if overlapDetected > threshold: # Si supera un threshold, este polígono al carrer
        f.write(str(idx)+"\n")
        break
  
###########
# Calcula el porcentaje de overlap de un polígono respecto a otro
###########
def getOverlapPercentage(p1, p2):
  overlapDetected   = p2.intersection(p1).area / p1.area * 100
  return overlapDetected

###########
# Transforma un array a un polígono en un EPSG determinado
###########
def polygonToWGS84(originArray, originEPSG, destEPSG=4326):
  if originEPSG == destEPSG:
    transformed = []
    for point in originArray:
      transformed.append([point[0], point[1]])
  else:
    # Extraer el EPSG que modela las coordenadas
    source = osr.SpatialReference()
    source.ImportFromEPSG(int(originEPSG))
    # Transformación a WGS84
    wgs84 = osr.SpatialReference()
    wgs84.ImportFromEPSG(destEPSG)
    transSourWGS = osr.CoordinateTransformation(source, wgs84)
    transWGSSour = osr.CoordinateTransformation(wgs84, source)
    
    # transformando los puntos de los recortes a hacer
    transformed = []
    for point in originArray:
      xAI, yAI, _ = transSourWGS.TransformPoint(point[0], point[1]) # arriba izq v2
      transformed.append([xAI, yAI])
  b = Polygon(transformed)
  return b

###########
# 
###########
def SHPgetPolygons(SHPName, SHPExtension=".shp"):
  polyList = []
  with fiona.open(SHPName+SHPExtension) as polyShp:
    if len(polyShp) == 1:  # Es multiPolygon
      #print("Es multipolygon")
      multiPol = polyShp[0]['geometry']['coordinates']
      #print("multiPol", len(multiPol))
      newPoly = []

      for b in multiPol:
        #print("b", len(b))
        newPoly2 = []
        for c in b:
          #print("c", len(c))
          newPoly2.append( c )
          #for d in c:  # Aqui ya siempre son 2, tuplas XY
          #  print("d", len(d))   
        newPoly.append( newPoly2 )
      polyList.append( newPoly )

    else: # Es polygon
      for poly in polyShp:
        polyGeom = poly['geometry']['coordinates']
        polyList.append(polyGeom[0])
      #return polyList
  
  return polyList

In [ ]:
cleanSHPOverlappingBuildingCSV("Train", "Train/", "BuildingsWGS84", "To_Clean_LiDAR_Detections/")
cleanSHPCSVtoSHP("Train", "Train/", "DeletedBuildingsDetections.txt", "Train_NoBuildings")

cleanSHPOverlappingRoadCSV("Train_NoBuildings", "Train/", "RoadsEPSG3345", "To_Clean_LiDAR_Detections/")
cleanSHPCSVtoSHP("Train_NoBuildings", "Train/", "DeletedRoadsDetections.txt", "Train_NoBuildings_NoRoads")

In [ ]:
cleanSHPOverlappingBuildingCSV("Example", "Example/", "BuildingsWGS84", "To_Clean_LiDAR_Detections/")
cleanSHPCSVtoSHP("Example", "Example/", "DeletedBuildingsDetections.txt", "Example_NoBuildings")

cleanSHPOverlappingRoadCSV("Example_NoBuildings", "Example/", "RoadsEPSG3345", "To_Clean_LiDAR_Detections/")
cleanSHPCSVtoSHP("Example_NoBuildings", "Example/", "DeletedRoadsDetections.txt", "Example_NoBuildings_NoRoads")

# SHP LiDAR detections related in a CSV with each pixel of the TIF image

In [ ]:
# TENEMOS UN SHP CON MILES DE RECTANGULOS EN LAT LON (EN UN SISTEMA DE COORDENDAS CONCRETO. WGS84??).
# LOS CONVERTIMOS A PIXELES EN LA ORTOFOTO RECORTADA DE COLME
# Pasamos las detecciones del LiDAR a un CSV para entrenar la ortofoto seleccionada
#EJ. PARA EL ARBOL EN LAT LONG X, LE CORRESPONDE EN LA IMAGEN DADA EL PIXEL Y
csv_train = "Train/Train_CSV_annotations/CSV_ALL_ORTHOPHOTO_annotations"
SHPtoCSV("Train/Train_NoBuildings_NoRoads", tif_image, csv_train)


# Train

In [ ]:
###################
def cropsDeepForest(TIFName, CSVName, trainTest, nameOutputModel, TIFExtension=".tif", CSVExtension=".csv", modelExtension=".h5"):
    # Lectura del CSV original
    dataset_annotations = pd.read_csv(CSVName+CSVExtension)
    # División de la imagen completa en trozos de 400x400 para entrenar
    cropsFolder = 'Train/Train_CSV_annotations/CROPS,NET_INPUTS'
    print(TIFName+TIFExtension)
    train_annotations = preprocess.split_raster(CSVName+CSVExtension, path_to_raster=TIFName+TIFExtension, 
                                                patch_size=400, base_dir=cropsFolder, patch_overlap=0.05)
    print("División en imágenes 400x400 completada")
    return train_annotations

def trainDeepForest(train_annotations, CSVName, trainTest, nameOutputModel, TIFExtension=".tif", CSVExtension=".csv", modelExtension=".h5"):
    # División del CSV completo en Train-Test para que aprenda la red
    image_dir = train_annotations.image_path.unique()
    print(image_dir)
    test_dir  = np.random.choice(image_dir, trainTest[1]) # Porcentaje test
    test_annotations  = train_annotations.loc[ train_annotations.image_path.isin(test_dir)]
    train_annotations = train_annotations.loc[~train_annotations.image_path.isin(test_dir)]
    #train_annotations.head() # impresion principio tabla
    #print("There are {} training crown annotations".format(train_annotations.shape[0]))
    #print("There are {} test crown annotations".format(test_annotations.shape[0]))
    print("División CSV train-test completada")

    cropsFolder = 'Train/Train_CSV_annotations/CROPS,NET_INPUTS/'
    # Genera ambos CSV
    filenameTrain = cropsFolder+"train"+nameOutputModel+CSVExtension
    filenameTest  = cropsFolder+"test"+nameOutputModel+CSVExtension
    train_annotations.to_csv(filenameTrain, index=False, header=False)
    test_annotations.to_csv( filenameTest,  index=False, header=False)

    ##### Creacion del modelo
    model = deepforest.deepforest() # Crea un modelo nuevo
    model.config["gpus"] = 1 # Para utilizar la GPU de Google Colab (activarla first)
    model.config["epochs"] = 5
    model.config["save-snapshot"] = True
    print("A generar modelo")
    model.train(annotations=filenameTrain, input_type="fit_generator") #ENTRENAMIENTO MODELO CON CONJUNTO DE ENTRENAMIENTO
    print("Modelo entrenado, guardando")
    #model.evaluate_generator(filenameTrain) #EVALUACIÓN DEL NUEVO MODELO MODELO: ACIERTO 60%
    model.model.save('Model/'+nameOutputModel+modelExtension) #GUARDAMOS EL MODELO

# guardar las anotaciones en CSV
def annotationsToCSV(name, train_annotations):
  train_annotations.to_csv(name+'_annotations.csv')

# cargar las anotaciones en CSV en el mismo objeto Pandas Dataframe
def loadCSVannotations(name):
  aa = pd.read_csv(name+'_annotations.csv')
  aa = aa.drop(aa.columns[[0]], axis=1)
  return aa

In [ ]:
# Entrenando campusNotBad
modelName='model'
#haveCSV = False
haveCSV = True
if haveCSV == True: # Carga las anotaciones directamente de un CSV, en vez de calcularlas y hacer todos los crop de nuevo
  train_annotations = loadCSVannotations('Train/Train_CSV_annotations/CSV_ORTHOPHOTO_CROPS')
else:
  Image.MAX_IMAGE_PIXELS = 337351030
  train_annotations = cropsDeepForest(tif_image, 'Train/Train_CSV_annotations/CSV_ALL_ORTHOPHOTO_annotations', [70, 30], modelName)
  annotationsToCSV('Train/Train_CSV_annotations/CSV_ORTHOPHOTO_CROPS', train_annotations) # Guarda las anotaciones

FileNotFoundError: ignored

In [ ]:
#print(train_annotations.image_path)
trainDeepForest(train_annotations, csv_train, [70, 30], modelName)

# Predict

In [ ]:
modelName='model'
# Cargar modelo entrenado
model = deepforest.deepforest(saved_model='Model/'+modelName+'.h5')

#PRREDICCION CON EL NUEVO MODELO
trained_model_boxes = model.predict_tile("Orthophoto_University_Example.tif", return_plot=True)

#VISUALIZACION RESULTADOS, CLARAMENTE MEJORA RESPECTO AL MODELO DEFAULT, PARA SOLO TENER 1500 INSTANCIAS Y 1 EPOCA ESTA MUY BIEN
fig = plt.figure(figsize=(20,20))
plt.imshow(trained_model_boxes)
plt.show()

Reading config file: /usr/local/lib/python3.7/dist-packages/deepforest/data/deepforest_config.yml
Loading saved model


OSError: ignored

In [ ]:
# Predicción
trained_model_boxes = model.predict_tile("Orthophoto_University_Example.tif", return_plot=False, patch_overlap=0.2,iou_threshold=0.2)
# Create SHP
deepForestToSHP(trained_model_boxes, TIFName="Orthophoto_University_Example", SHPOutputName='Example/net_detections')
# Create CSV
SHPtoCSV(SHPName='Example/net_detections', TIFName="Orthophoto_University_Example",CSVName='Example/net_detections_CSV_annotations')

NameError: ignored

# Analysis of the results

In [ ]:

def calculatePolygonArea(polygonVec):
  geod = Geod(ellps="WGS84") # specify a named ellipsoid
  poly = Polygon(polygonVec)
  area = abs(geod.geometry_area_perimeter(poly)[0])
  print('# Geodesic area: {:.3f} m^2'.format(area))
  return area

def calculateSHPAreas(SHPName, TIFName, CSVName, SHPExtension=".shp", TIFExtension=".tif", CSVExtension=".csv"):
  SHP_ESPG = SHPgetESPG(SHPName, SHPExtension)
  polyList = SHPgetPolygons(SHPName, SHPExtension)

  arrArea = []
  for poly in polyList:
    a = calculatePolygonArea(poly)
    arrArea.append(a)
  
  return arrArea

In [ ]:

raw_symbols = SymbolValidator().values

symbols = []
for i in range(0,len(raw_symbols),3):
    symbols.append(raw_symbols[i])
colors = px.colors.qualitative.Dark24

In [ ]:
############################
# Extraer las dimensiones medias de cada bounding-box

# Create CSV
SHPtoCSV(SHPName='Example/Test_NoBuildings_NoRoads', TIFName="Orthophoto_University_Example",CSVName='Example/Test_NoBuildings_NoRoads_CSV_annotations')

shp_test = "Example/Test_NoBuildings_NoRoads" # LiDAR
csv_test = "Example/Test_NoBuildings_NoRoads_CSV_annotations"
tif_image = "Orthophoto_University_Example"
lidarAreas = calculateSHPAreas(shp_test, tif_image, csv_test)

shp_result = "Example/net_detections"
csv_result = "Example/net_detections_CSV_annotations"
imageAreas = calculateSHPAreas(shp_result, tif_image, csv_result)

SHPtoCSV


DriverError: ignored

In [ ]:
colors = px.colors.qualitative.T10

############## CADA SEGMENTATOR POR SEPARADO
fig = go.Figure()

#aa = ["LiDAR"] * len(a)
fig.add_trace(
    go.Box(
        name="LiDAR",
        x=lidarAreas,
        #boxmean='sd', # True represent mean; 'sd' represent mean and standard deviation
    )
)
#aa = ["Imagery"] * len(a)
fig.add_trace(
    go.Box(
        name="Imagery",
        x=imageAreas,
        #boxmean='sd', # True represent mean; 'sd' represent mean and standard deviation
    )
)

fig.update_layout(
    legend=dict(
        traceorder="normal",
        font=dict(
            size=14,
            color="black"
        ),
        bordercolor="Black",
        borderwidth=1,
        xanchor = "center", # Para centrarla horizontalmente
        x = 0.5,            # Para centrarla horizontalmente
        orientation="h",    # Para centrarla horizontalmente
    ),
    autosize=False,
    width=1400,
    height=450,
    # figure layout adjustments
    #boxmode='group',
    #xaxis_tickangle=0,
)
fig.update_xaxes(range=[0, 100])
fig.show() # Para que salga el resultado

outputFile         = F"./percentOverlap.svg" # svg                   # output file
pio.write_image(fig, outputFile) # Se guarda el diagrama en la carpeta de Drive / local